In [1]:
# DEV_Stocks_Data_Calculate_Based_on_Daily_004_Thread.ipynb
# Create by GF 2025-11-27 22:32

In [2]:
import json
import sys
import threading
# ..................................................
import pandas      # Pandas 2.3.3
import sqlalchemy  # SQLAlchemy 2.0.36
# ..................................................
import APP_Stocks_DataFrame_Calculator
# ..................................................
import PY3_PostgreSQL9_By_Psycopg2
import PY3_Text_Progress_Bar
import PY38_Array_Spliter

In [3]:
# 读取 Finance Indicator Calculate Configuration (JSON 文件)
# ..................................................
FILE_Object = open("./APP_Finance_Calculate_Configuration.json", mode = 'r', encoding = "utf-8")
TEXT_Readed = FILE_Object.read()
FILE_Object.close()
DICT_Config = json.loads(TEXT_Readed)
# ..................................................
db_host     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("host", "unknow_host")
db_port     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("port", "unknow_port")
db_name     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("init_database", "unknow_db_name")
db_user     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("user", "unknow_user")
db_password = DICT_Config.get("postgresql", {}).get("16.10", {}).get("password", "unknow_password")
# ..................................................
stocks_mixed_db_table = DICT_Config.get("options", {}).get("stocks", {}).get("mixed", {}).get("db_table", "unkonw_db_table")
stocks_mixed_adjusted = DICT_Config.get("options", {}).get("stocks", {}).get("mixed", {}).get("adjusted", "unkonw_adjusted")

In [4]:
PGSQL             = PY3_PostgreSQL9_By_Psycopg2.PY3_PostgreSQL9_By_Psycopg2()
PGSQL.DB_HOST     = db_host
PGSQL.DB_DATABASE = db_name
PGSQL.DB_USER     = db_user
PGSQL.DB_PASSWORD = db_password
# ..................................................
Text_Progress_Bar = PY3_Text_Progress_Bar.PY3_Text_Progress_Bar()
Array_Spliter     = PY38_Array_Spliter.PY38_Array_Spliter()

In [5]:
SQL_FILTERING:str = "POSITION('股票' IN memo) > 0 AND POSITION('日数据' IN memo) > 0 AND POSITION('%s' IN memo) > 0" % stocks_mixed_adjusted
SQL_STATEMENT:str = "SELECT * FROM %s WHERE %s;" % (stocks_mixed_db_table, SQL_FILTERING)
df_stocks_daily_bfq = PGSQL.Query(SQL_STATEMENT)
df_stocks_daily_bfq = df_stocks_daily_bfq.rename(columns = {"ts_code": "code", "pct_chg": "chg_pct", "vol": "volume"})
df_stocks_daily_bfq

,id,unique,code,time,open,high,low,close,pre_close,change,chg_pct,volume,amount,memo
0,978374,stock-300135-SZ-daily-20250930-bfq,300135.SZ,2025-09-30,4.4100,4.4300,4.3500,4.3600,4.3900,-0.0300,-0.6834,198088.0000,86594.1000,股票/日数据/不复权
1,978375,stock-300136-SZ-daily-20250930-bfq,300136.SZ,2025-09-30,25.8500,26.2400,25.4100,25.8000,25.7800,0.0200,0.0776,291497.0000,749882.0000,股票/日数据/不复权
2,978376,stock-300137-SZ-daily-20250930-bfq,300137.SZ,2025-09-30,6.6000,6.7900,6.5500,6.7500,6.6300,0.1200,1.8100,54999.6000,36986.9000,股票/日数据/不复权
3,978377,stock-300138-SZ-daily-20250930-bfq,300138.SZ,2025-09-30,13.0000,13.2300,12.9300,13.1100,12.9600,0.1500,1.1574,49482.6000,64794.7000,股票/日数据/不复权
4,978378,stock-300139-SZ-daily-20250930-bfq,300139.SZ,2025-09-30,28.0500,28.8000,27.7500,27.8100,27.6700,0.1400,0.5060,654474.0000,1846360.0000,股票/日数据/不复权
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210410,1013661,stock-603813-SH-daily-20251014-bfq,603813.SH,2025-10-14,25.1500,25.1500,24.2400,24.5900,24.0500,0.5400,2.2453,11691.0000,28762.4000,股票/日数据/不复权
1210411,1013662,stock-603815-SH-daily-20251014-bfq,603815.SH,2025-10-14,14.2500,14.8000,13.4000,13.5500,14.2500,-0.7000,-4.9123,279273.0000,386475.2188,股票/日数据/不复权
1210412,1013663,stock-603816-SH-daily-20251014-bfq,603816.SH,2025-10-14,29.9600,30.5900,29.7200,30.0400,29.9600,0.0800,0.2670,47381.8000,142708.9531,股票/日数据/不复权
1210413,1013664,stock-603817-SH-daily-20251014-bfq,603817.SH,2025-10-14,6.4400,6.4700,6.4000,6.4500,6.4200,0.0300,0.4673,98407.5000,63351.4000,股票/日数据/不复权


In [14]:
def Finance_Calculate_Stocks_Based_on_Daily(
        Pandas_DataFrame_for_Stocks_Daily,
        DB_TABLE_Stocks_Data_Daily:str            = "tushare_api_cache_en_us",
        DB_TABLE_Stocks_Data_Daily_Indicator:str  = "tushare_api_cache_en_us_indicator",
        DB_TABLE_Stocks_Data_Weekly:str           = "stocks_en_us_daily_to_weekly",
        DB_TABLE_Stocks_Data_Weekly_Indicator:str = "stocks_en_us_daily_to_weekly_indicator",
        DB_TABLE_Stocks_Data_Weekly_Interim:str   = "stocks_en_us_daily_to_weekly_interim",
        Stocks_Data_Adjusted = "bfq"):

    Adjusted:str = "不复权"
    if (Stocks_Data_Adjusted == "bfq"): Adjusted = "不复权"
    if (Stocks_Data_Adjusted == "hfq"): Adjusted = "后复权"
    if (Stocks_Data_Adjusted == "qfq"): Adjusted = "前复权"

    ########################################
    ## Calculating Stocks Daily Indicator ##
    ########################################

    df = Pandas_DataFrame_for_Stocks_Daily.copy()
    # ..............................................
    Tasks_Target         = list(df["code"].drop_duplicates())
    Tasks_Target_Splited = Array_Spliter.Split_Array_into_Multi_Segment_Array_By_Equally(Array = Tasks_Target, Segments_Count = 4)
    # ..............................................
    Task_001_APP_Stocks_DataFrame_Calculator = APP_Stocks_DataFrame_Calculator.APP_Stocks_DataFrame_Calculator()
    Task_002_APP_Stocks_DataFrame_Calculator = APP_Stocks_DataFrame_Calculator.APP_Stocks_DataFrame_Calculator()
    Task_003_APP_Stocks_DataFrame_Calculator = APP_Stocks_DataFrame_Calculator.APP_Stocks_DataFrame_Calculator()
    Task_004_APP_Stocks_DataFrame_Calculator = APP_Stocks_DataFrame_Calculator.APP_Stocks_DataFrame_Calculator()
    # ..............................................
    Task_001_APP_Stocks_DataFrame_Calculator.Init(df[df["code"].isin(Tasks_Target_Splited["multi_segment_array"][0]) == True])
    Task_002_APP_Stocks_DataFrame_Calculator.Init(df[df["code"].isin(Tasks_Target_Splited["multi_segment_array"][1]) == True])
    Task_003_APP_Stocks_DataFrame_Calculator.Init(df[df["code"].isin(Tasks_Target_Splited["multi_segment_array"][2]) == True])
    Task_004_APP_Stocks_DataFrame_Calculator.Init(df[df["code"].isin(Tasks_Target_Splited["multi_segment_array"][3]) == True])

    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator...\n""")
    ITERATOR_MAXIDX:int = sum([Task_001_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_002_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_003_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_004_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX])
    ITERATOR_MOVING:int = sum([Task_001_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_002_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_003_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_004_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING])
    Total:int = ITERATOR_MAXIDX + 1
    Count:int = 4
    while (ITERATOR_MOVING <= ITERATOR_MAXIDX):
        if (Count == Total or Count % 100 == 0):
            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
            sys.stdout.flush()
        threads:list = []
        threads.append( threading.Thread(target = Task_001_APP_Stocks_DataFrame_Calculator.Next) )
        threads.append( threading.Thread(target = Task_002_APP_Stocks_DataFrame_Calculator.Next) )
        threads.append( threading.Thread(target = Task_003_APP_Stocks_DataFrame_Calculator.Next) )
        threads.append( threading.Thread(target = Task_004_APP_Stocks_DataFrame_Calculator.Next) )
        for t in threads:
            t.start()
        for t in threads:
            t.join()
        ITERATOR_MAXIDX = sum([Task_001_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_002_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_003_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX,
                               Task_004_APP_Stocks_DataFrame_Calculator.ITERATOR_MAXIDX])
        ITERATOR_MOVING = sum([Task_001_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_002_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_003_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING,
                               Task_004_APP_Stocks_DataFrame_Calculator.ITERATOR_MOVING])
        # ..........................................
        Count = Count + 4
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator, Completed!\n""")

#    #############################################################################
#    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Daily_Indicator>", Writing Data ##
#    #############################################################################
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Daily_Indicator}", Writing Data...\n""")
#    df = pandas.DataFrame(JSON_Records_Stocks_Daily_Indicator)
#    # ..............................................
#    PGSQL.TRUNCATE_TABLE(DB_TABLE = DB_TABLE_Stocks_Data_Daily_Indicator, PRIMARY_KEY = "id")
#    PGSQL.INSERT_ITERATOR_INIT_BY_Pandas_DataFrame(Pandas_DataFrame = df, DB_TABLE = DB_TABLE_Stocks_Data_Daily_Indicator)
#    while (PGSQL.INSERT_ITERATOR_MOVING <= PGSQL.INSERT_ITERATOR_MAXIDX):
#        Count:int = PGSQL.INSERT_ITERATOR_MOVING + 1
#        Total:int = PGSQL.INSERT_ITERATOR_MAXIDX + 1
#        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#        PGSQL.INSERT_ITERATOR_NEXT()
#    sys.stdout.write(f"""\n""")
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Daily_Indicator}", Writing Completed!\n""")
#
#    ############################################
#    ## Calculating Stocks Data Weekly Interim ##
#    ############################################
#
#    df = Pandas_DataFrame_for_Stocks_Daily.copy()
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim...\n""")
#    df = Stocks_DataFrame_Preprocessing.Before_Stocks_Data_Converting_Daily_to_Weekly(df)
#    # ..............................................
#    JSON_Records_Stocks_Weekly_Interim:list  = []
#    # ..............................................
#    Total = df["id"].count()
#    Count = 1
#    # ..............................................
#    for Idx, Row in df.iterrows():
#        ID            = Row["id"]
#        ROW_NUM       = Row["row_num"]        # Temporary Column for Calculation.
#        Time          = Row["time"]
#        Week_Num      = Row["week_num"]
#        ISO_8601_Week = Row["iso_8601_week"]  # Temporary Column for Calculation.
#        Code          = Row["code"]
#        Open          = Row["open"]
#        High          = Row["high"]
#        Low           = Row["low"]
#        Close         = Row["close"]
#        Change        = Row["change"]
#        Volume        = Row["volume"]
#        # ..........................................
#        if (Count == Total or Count % 100 == 0):
#            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#            sys.stdout.flush()
#        # ..........................................
#        JSON_Records_Stocks_Weekly_Interim.append({
#            "id":            ID,
#            "row_num":       ROW_NUM,
#            "time":          Stocks_Data_Conv_Daily_to_Weekly.WK_Time(ROW_NUM = ROW_NUM, DY_Time = Time),
#            "week_num":      Week_Num,
#            "iso_8601_week": ISO_8601_Week,
#            "code":          Code,
#            "open":          Stocks_Data_Conv_Daily_to_Weekly.WK_Open(ROW_NUM = ROW_NUM, DY_Open = Open),
#            "high":          Stocks_Data_Conv_Daily_to_Weekly.WK_High(ROW_NUM = ROW_NUM, DY_High = High),
#            "low":           Stocks_Data_Conv_Daily_to_Weekly.WK_Low(ROW_NUM = ROW_NUM, DY_Low = Low),
#            "close":         Stocks_Data_Conv_Daily_to_Weekly.WK_Close(ROW_NUM = ROW_NUM, DY_Close = Close),
#            "change":        Stocks_Data_Conv_Daily_to_Weekly.WK_Change(ROW_NUM = ROW_NUM, DY_Change = Change),
#            "volume":        Stocks_Data_Conv_Daily_to_Weekly.WK_Volume(ROW_NUM = ROW_NUM, DY_Volume = Volume)
#        })
#        # ..........................................
#        Count = Count + 1
#    sys.stdout.write(f"""\n""")
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim, Completed!\n""")
#
#    ############################################################################
#    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly_Interim>", Writing Data ##
#    ############################################################################
#
#    if (DB_TABLE_Stocks_Data_Weekly_Interim != "<none>"):
#        sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Interim}", Writing Data...\n""")
#        df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Interim)
#        df = df.drop(["row_num", "iso_8601_week"], axis = 1)  # Delete The Temporary Column for Calculation.
#        # ..........................................
#        PGSQL.TRUNCATE_TABLE(DB_TABLE = DB_TABLE_Stocks_Data_Weekly_Interim, PRIMARY_KEY = "id")
#        PGSQL.INSERT_ITERATOR_INIT_BY_Pandas_DataFrame(Pandas_DataFrame = df, DB_TABLE = DB_TABLE_Stocks_Data_Weekly_Interim)
#        while (PGSQL.INSERT_ITERATOR_MOVING <= PGSQL.INSERT_ITERATOR_MAXIDX):
#            Count:int = PGSQL.INSERT_ITERATOR_MOVING + 1
#            Total:int = PGSQL.INSERT_ITERATOR_MAXIDX + 1
#            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#            PGSQL.INSERT_ITERATOR_NEXT()
#        sys.stdout.write(f"""\n""")
#        sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Interim}", Writing Completed!\n""")
#
#    ############################################
#    ## Aggregating Stocks Data Weekly Interim ##
#    ############################################
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim...\n""")
#    df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Interim)
#    df["cum_max"] = df[["iso_8601_week", "row_num"]].groupby("iso_8601_week", as_index = False).cummax()["row_num"]
#    df = df.query(""" row_num == cum_max """)
#    df["memo"   ] = f"股票/周数据/{Adjusted}"
#    df = df.drop(["row_num", "iso_8601_week", "cum_max"], axis = 1)  # Delete The Temporary Column for Calculation.
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim, Completed!\n""")
#
#    ####################################################################
#    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly>", Writing Data ##
#    ####################################################################
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly}", Writing Data...\n""")
#    PGSQL.TRUNCATE_TABLE(DB_TABLE = DB_TABLE_Stocks_Data_Weekly, PRIMARY_KEY = "id")
#    PGSQL.INSERT_ITERATOR_INIT_BY_Pandas_DataFrame(Pandas_DataFrame = df, DB_TABLE = DB_TABLE_Stocks_Data_Weekly)
#    while (PGSQL.INSERT_ITERATOR_MOVING <= PGSQL.INSERT_ITERATOR_MAXIDX):
#        Count:int = PGSQL.INSERT_ITERATOR_MOVING + 1
#        Total:int = PGSQL.INSERT_ITERATOR_MAXIDX + 1
#        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#        PGSQL.INSERT_ITERATOR_NEXT()
#    sys.stdout.write(f"""\n""")
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly}", Writing Completed!\n""")
#
#    ########################################
#    ## Calculating Stocks Weekly Indicator ##
#    ########################################
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Indicator...\n""")
#    df = Stocks_DataFrame_Preprocessing.Before_Calculating_Stocks_Weekly_Indicators(df)
#    # ..............................................
#    JSON_Records_Stocks_Weekly_Indicator:list = []
#    # ..............................................
#    Total = df["id"].count()
#    Count = 1
#    # ..............................................
#    for Idx, Row in df.iterrows():
#        ID            = Row["id"]
#        ROW_NUM       = Row["row_num"]
#        High          = Row["high"]
#        Low           = Row["low"]
#        Close         = Row["close"]
#        # ..........................................
#        if (Count == Total or Count % 100 == 0):
#            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#            sys.stdout.flush()
#        # ..........................................
#        EMA12       = Fin_Ind_EMA_for_012.EMA(ROW_NUM = ROW_NUM, Period = 12, Close = Close)
#        EMA26       = Fin_Ind_EMA_for_026.EMA(ROW_NUM = ROW_NUM, Period = 26, Close = Close)
#        MACD_DIF    = Fin_Ind_MACD.MACD_DIF(EMA12 = EMA12, EMA26 = EMA26)
#        MACD_DEA    = Fin_Ind_MACD.MACD_DEA(ROW_NUM = ROW_NUM, MACD_DIF = MACD_DIF)
#        MACD_STICK  = Fin_Ind_MACD.MACD_STICK(MACD_DIF = MACD_DIF, MACD_DEA = MACD_DEA)
#        KDJ_K       = Fin_Ind_KDJ.KDJ_K(ROW_NUM = ROW_NUM, RSV_Prd = 9, K_Prd = 3, High = High, Low = Low, Close = Close)
#        KDJ_D       = Fin_Ind_KDJ.KDJ_D(ROW_NUM = ROW_NUM, RSV_Prd = 9, D_Prd = 3, K_Val = KDJ_K)
#        KDJ_J       = Fin_Ind_KDJ.KDJ_J(K_Val = KDJ_K, D_Val = KDJ_D)
#        ETG_TRS     = Fin_Etg_Theory.Top_Reversal_Shape(ROW_NUM = ROW_NUM, Input_UpperEdge = High, Input_LowerEdge = Low)
#        ETG_BRS     = Fin_Etg_Theory.Bottom_Reversal_Shape(ROW_NUM = ROW_NUM, Input_UpperEdge = High, Input_LowerEdge = Low)
#        ETG_T_GROUP = Fin_Etg_Theory.Top_Reversal_Shape_s_Group_Top(ROW_NUM = ROW_NUM, Input_UpperEdge = High, Input_LowerEdge = Low)
#        ETG_B_GROUP = Fin_Etg_Theory.Bottom_Reversal_Shape_s_Group_Bottom(ROW_NUM = ROW_NUM, Input_UpperEdge = High, Input_LowerEdge = Low)
#        # ..........................................
#        JSON_Records_Stocks_Weekly_Indicator.append({
#            "id":          ID,
#            "sma5":        Fin_Ind_SMA_for_005.SMA(ROW_NUM = ROW_NUM, Period = 5, Close = Close),
#            "sma10":       Fin_Ind_SMA_for_010.SMA(ROW_NUM = ROW_NUM, Period = 10, Close = Close),
#            "ema12":       EMA12,
#            "ema26":       EMA26,
#            "macd_dif":    MACD_DIF,
#            "macd_dea":    MACD_DEA,
#            "macd_stick":  MACD_STICK,
#            "kdj_k":       KDJ_K,
#            "kdj_d":       KDJ_D,
#            "kdj_j":       KDJ_J,
#            "etg_trs":     ETG_TRS,
#            "etg_brs":     ETG_BRS,
#            "etg_t_group": ETG_T_GROUP,
#            "etg_b_group": ETG_B_GROUP
#        })
#        # ..........................................
#        Count = Count + 1
#    sys.stdout.write(f"""\n""")
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Indicator, Completed!\n""")
#
#    #############################################################################
#    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly_Indicator>", Writing Data ##
#    #############################################################################
#
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Indicator}", Writing Data...\n""")
#    df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Indicator)
#    # ..............................................
#    PGSQL.TRUNCATE_TABLE(DB_TABLE = DB_TABLE_Stocks_Data_Weekly_Indicator, PRIMARY_KEY = "id")
#    PGSQL.INSERT_ITERATOR_INIT_BY_Pandas_DataFrame(Pandas_DataFrame = df, DB_TABLE = DB_TABLE_Stocks_Data_Weekly_Indicator)
#    while (PGSQL.INSERT_ITERATOR_MOVING <= PGSQL.INSERT_ITERATOR_MAXIDX):
#        Count:int = PGSQL.INSERT_ITERATOR_MOVING + 1
#        Total:int = PGSQL.INSERT_ITERATOR_MAXIDX + 1
#        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
#        PGSQL.INSERT_ITERATOR_NEXT()
#    sys.stdout.write(f"""\n""")
#    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Indicator}", Writing Completed!\n""")

In [ ]:
Finance_Calculate_Stocks_Based_on_Daily(df_stocks_daily_bfq)

In [7]:
import time

In [8]:
Time_0 = time.time()
Finance_Calculate_Stocks_Based_on_Daily(df_stocks_daily_bfq)
Time_1 = time.time()
# ..................................................
Time_X:dict = {    "basic_second": (Time_1 - Time_0)}
Time_X.update({"remainder_second": (Time_X["basic_second"] % 60)})
Time_X.update({  "integer_minute": (Time_X["basic_second"] - Time_X["remainder_second"]) / 60})
# ..................................................
sys.stdout.write(f"""[DEBUG] Time Consumption: {Time_X["integer_minute"]} Minute {Time_X["remainder_second"]} Second\n""")

[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1183227/1183227
[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator, Completed!
[DEBUG] PROCESSING: Towards DB_TABLE "tushare_api_cache_en_us_indicator", Writing Data...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1183227/1183227
[DEBUG] PROCESSING: Towards DB_TABLE "tushare_api_cache_en_us_indicator", Writing Completed!
[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1183227/1183227
[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim, Completed!
[DEBUG] PROCESSING: Towards DB_TABLE "stocks_en_us_daily_to_weekly_interim", Writing Data...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1183227/1183227
[DEBUG] PROCESSING: Towards DB_TABLE "stocks_en_us_daily_to_weekly_interim", Writing Completed!
[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim...
[DEBUG] PR

63

In [5]:
import pandas

In [6]:
df = pandas.DataFrame({"A": ["Hello", "World", 3]})

In [7]:
Array_Spliter.Split_Array_into_Multi_Segment_Array_By_Equally(list(df["A"]), Segments_Count = 2, Allocate_Remaining = 1)

{'multi_segment_array': [['Hello', 'World'], [3]], 'remaining_array': []}

In [20]:
Tasks_Target         = list(df_stocks_daily_bfq["code"].drop_duplicates())
Tasks_Target_Splited = Array_Spliter.Split_Array_into_Multi_Segment_Array_By_Equally(Array = Tasks_Target, Segments_Count = 4)
len(Tasks_Target)

5726

In [21]:
len(Tasks_Target_Splited["multi_segment_array"][0]) + \
len(Tasks_Target_Splited["multi_segment_array"][1]) + \
len(Tasks_Target_Splited["multi_segment_array"][2]) + \
len(Tasks_Target_Splited["multi_segment_array"][3])

5726

In [22]:
len(Tasks_Target_Splited["multi_segment_array"][0])

1432

In [23]:
len(Tasks_Target_Splited["multi_segment_array"][0])

1432

In [30]:
Task_001_APP_Stocks_DataFrame_Calculator = APP_Stocks_DataFrame_Calculator.APP_Stocks_DataFrame_Calculator()
Task_001_APP_Stocks_DataFrame_Calculator.Init(df_stocks_daily_bfq[df_stocks_daily_bfq["code"].isin(Tasks_Target_Splited["multi_segment_array"][0]) == True])

316936

In [31]:
Task_001_APP_Stocks_DataFrame_Calculato

-1

In [8]:
sum([1,2,3])

6